In [1]:
import pandas as pd
from sklearn import __version__ as sklearn_version
from packaging import version
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error

# Load the dataset
file_path = "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/combinedWeatherValues.csv"  # Replace with the actual path
data = pd.read_csv(file_path)

# Convert 'date' column to datetime
data['date'] = pd.to_datetime(data['date'])

# Define emissions parameters and load
emissions_params = ['SO2TONS', 'NOXTONS', 'COTONS']
load_param = 'LOADMWBA'

# Separate emissions and load data
emissions_data = data[data['Parameter'].isin(emissions_params)]
load_data = data[data['Parameter'] == load_param]

# Merge emissions and load data on date and source (use a left join to retain emissions data)
merged_data = pd.merge(
    emissions_data,
    load_data,
    on=["date", "Source"],
    suffixes=("_emission", "_load"),
    how="left"
)

# Calculate emissions/load only for rows where both emissions and load are available
merged_data["Emissions_Load"] = merged_data["Value_emission"] / merged_data["Value_load"]

# Drop rows with NaN in Emissions_Load
merged_data = merged_data.dropna(subset=["Emissions_Load"])

# Define predictors and target
predictors = ['tavg_emission', 'tmin_emission', 'tmax_emission', 'prcp_emission',
              'snow_emission', 'wdir_emission', 'wspd_emission', 'pres_emission']
categorical_features = ['Source', 'Parameter_emission', 'Units_emission']
target = 'Emissions_Load'

# Dynamic OneHotEncoder
if version.parse(sklearn_version) >= version.parse("1.2"):
    one_hot_encoder = OneHotEncoder(drop="first", sparse_output=False)  # For newer versions
else:
    one_hot_encoder = OneHotEncoder(drop="first")  # For older versions (before 1.2)

# Preprocess categorical and numeric features
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', one_hot_encoder, categorical_features),
        ('num', StandardScaler(), predictors)
    ]
)

# Prepare features and target
X = merged_data[predictors + categorical_features]
y = merged_data[target]

# Encode features
X_encoded = column_transformer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Train the Random Forest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on Test Data: {mae:.6f}")

# Function to find valid dates for prediction
def find_valid_dates(season_filter, num_days):
    if season_filter == 'peak':
        season_data = merged_data[merged_data['date'].dt.month.isin([5, 6, 7, 8])]  # Peak Season
    elif season_filter == 'off-peak':
        season_data = merged_data[merged_data['date'].dt.month.isin([1, 2, 3, 4, 9, 10, 11, 12])]  # Off-Peak Season
    else:
        season_data = merged_data

    unique_dates = season_data['date'].unique()
    return [str(date.date()) for date in unique_dates[:num_days]]

# Forecast emissions for multiple dates and analyze efficiency
def forecast_and_analyze_efficiency():
    # Select one Peak Season date and two Off-Peak Season dates
    peak_date = find_valid_dates('peak', num_days=1)
    off_peak_dates = find_valid_dates('off-peak', num_days=2)
    selected_dates = peak_date + off_peak_dates

    print(f"Selected Dates for Prediction: {selected_dates}")

    efficiency_results = {}
    results = []
    for unit in merged_data['Source'].unique():
        unit_results = []
        for date_to_predict in selected_dates:
            selected_day = merged_data[(merged_data['date'] == pd.Timestamp(date_to_predict)) &
                                        (merged_data['Source'] == unit)]
            if selected_day.empty:
                continue
            selected_day = selected_day.iloc[0]

            # Extract features for the selected day
            selected_day_features = pd.DataFrame([{
                **{col: selected_day[col] for col in predictors},
                **{col: selected_day[col] for col in categorical_features}
            }])

            # Transform features for prediction
            selected_day_encoded = column_transformer.transform(selected_day_features)
            predicted_emissions_load = model.predict(selected_day_encoded)[0]

            unit_results.append(predicted_emissions_load)

            # Collect actual values for comparison
            results.append({
                "Unit": unit,
                "Date": date_to_predict,
                "Predicted Emissions/Load": predicted_emissions_load,
                "Actual Emissions/Load": selected_day["Emissions_Load"]
            })

        # Calculate efficiency for each unit (average emissions/load)
        if unit_results:
            avg_efficiency = sum(unit_results) / len(unit_results)
            efficiency_results[unit] = avg_efficiency
            print(f"Average Emissions/Load for Unit {unit}: {avg_efficiency:.6f} tons/MW")

    # Identify the most efficient unit
    most_efficient_unit = min(efficiency_results, key=efficiency_results.get)
    print(f"\nThe most efficient unit is {most_efficient_unit} with an average Emissions/Load of "
          f"{efficiency_results[most_efficient_unit]:.6f} tons/MW.")

    # Display results
    for result in results:
        print(f"\nDate: {result['Date']} | Unit: {result['Unit']}")
        print(f"Predicted Emissions/Load: {result['Predicted Emissions/Load']:.6f} tons/MW")
        print(f"Actual Emissions/Load: {result['Actual Emissions/Load']:.6f} tons/MW")

# Run the forecasting and efficiency analysis
forecast_and_analyze_efficiency()


Mean Absolute Error on Test Data: 0.000176
Selected Dates for Prediction: ['2022-05-01', '2022-01-01', '2022-01-02']
Average Emissions/Load for Unit LAKE-1: 0.000034 tons/MW
Average Emissions/Load for Unit LAKE-2: 0.000048 tons/MW
Average Emissions/Load for Unit LAKE-3: 0.000043 tons/MW

The most efficient unit is LAKE-1 with an average Emissions/Load of 0.000034 tons/MW.

Date: 2022-05-01 | Unit: LAKE-1
Predicted Emissions/Load: 0.000026 tons/MW
Actual Emissions/Load: 0.000012 tons/MW

Date: 2022-01-01 | Unit: LAKE-1
Predicted Emissions/Load: 0.000028 tons/MW
Actual Emissions/Load: 0.000018 tons/MW

Date: 2022-01-02 | Unit: LAKE-1
Predicted Emissions/Load: 0.000049 tons/MW
Actual Emissions/Load: 0.000046 tons/MW

Date: 2022-05-01 | Unit: LAKE-2
Predicted Emissions/Load: 0.000048 tons/MW
Actual Emissions/Load: 0.000061 tons/MW

Date: 2022-01-01 | Unit: LAKE-3
Predicted Emissions/Load: 0.000038 tons/MW
Actual Emissions/Load: 0.000024 tons/MW

Date: 2022-01-02 | Unit: LAKE-3
Predicted Em